In [ ]:
import time, pickle, glob, os, sys, resource
from matching_functions import *

EEBO_SHINGLE_FOLDER = '/home/spenteco/0/eebo_shingled/'

all_shingles = []
shingle_lookup = {}
shingle_n = 0

f = open('/home/data/all_shingles.tsv', 'w', encoding='utf-8')

for n, path_to_eebo_pickle in enumerate(sorted(glob.glob(EEBO_SHINGLE_FOLDER + '*.pickle'))):
    
    if n % 1000 == 0:
        print('processing', n)
    
    tcp_id = path_to_eebo_pickle.split('/')[-1].split('.')[0]
    
    eebo_data = load_pickle_file(path_to_eebo_pickle)
    
    for k, v in eebo_data['shingles'].items():
        f.write(str(k) + '\t' + tcp_id + '\t' + str(v) + '\n')

f.close()

In [ ]:
!sort -T ~/0 /home/data/all_shingles.tsv > ~/0/SORTED.all_shingles.tsv

In [ ]:
inf = open('/home/spenteco/0/SORTED.all_shingles.tsv')
outf = open('/home/spenteco/0/GROUPED.all_shingles.tsv', 'w', encoding='utf-8')

key = ''
data = []
line_n = 0

for line in inf:
    if line.strip() > '':
        
        line_n += 1
        if line_n % 1000000 == 0:
            print(line_n, 'read')
        
        parts = line.strip().split('\t')
        
        if len(parts) == 3:
        
            k = parts[0]
            d = ';'.join(parts[1:])
        
            if k != key:
                if len(data) > 1:
                    outf.write(key + '\t' + '\t'.join(data) + '\n')
            
                data = []
                key = k
                
            data.append(d)
            
if len(data) > 1:
    outf.write(key + '\t' + '\t'.join(data) + '\n')
            
outf.close()          
inf.close()

In [ ]:
!sed -E 's/\t.+//g' ~/0/SORTED.all_shingles.tsv  | uniq -c | egrep -v '^\s+1\s' > ~/0/UNIQ.all_shingles.tsv 

egrep '[0-9]{5,}' UNIQ.all_shingles.tsv

In [ ]:
x = ['a', 'b', 'c', 'd']

test_pairs = []

for a in x:
    for b in x:
        if a != b:
            test_pairs.append(tuple(sorted([a, b])))
            
test_pairs = sorted(list(set(test_pairs)))

print(test_pairs)

test_pairs = []

for a in range(0, len(x) - 1):
    for b in range(a + 1, len(x)):
        if a != b:
            test_pairs.append(tuple(sorted([x[a], x[b]])))
            
test_pairs = sorted(list(set(test_pairs)))

print(test_pairs)

In [ ]:
from collections import defaultdict

inf = open('/home/spenteco/0/GROUPED.all_shingles.tsv')
#outf = open('/home/spenteco/0/PAIRED.all_shingle_matches.tsv', 'w', encoding='utf-8')

line_n = 0

tcp_ids_to_n = {}
matching_pairs = defaultdict(int)

for line in inf:
    if line.strip() > '':
        
        line_n += 1
        if line_n % 1000000 == 0:
            print(line_n, 'read')
        
        parts = line.strip().split('\t')
        
        tcp_ids_locations = {}
        
        for p in parts[1:]:
            tcp_id = p.split(';')[0]
            locations = p.split(';')[1]
            
            tcp_ids_locations[tcp_id] = locations
            
        tcp_ids = sorted(tcp_ids_locations.keys())
        
        for a in range(0, len(tcp_ids) - 1):
            for b in range(a + 1, len(tcp_ids)):
                
                matching_pairs[(tcp_ids[a]), tcp_ids[b]] += 1
                
                #outf.write(tcp_ids[a] + '\t' + \
                #           tcp_ids[b] + '\t' + \
                #           tcp_ids_locations[tcp_ids[a]] + '\t' + \
                #           tcp_ids_locations[tcp_ids[b]] + '\n' )
              
    if line_n > 10:
        break
    
            
#outf.close()          
inf.close()

print(len(matching_pairs))
n_gt_1 = 0
for k, v in matching_pairs.items():
    if v > 1:
        n_gt_1 += 1
        
print(n_gt_1)

sudo mkdir /mnt/ramdisk

sudo mount -t ramfs -o size=12G ramfs /mnt/ramdisk

sudo cp /home/spenteco/0/GROUPED.all_shingles.tsv /mnt/ramdisk

sudo umount /mnt/ramdisk

free -g

In [1]:
import time, glob, subprocess
from queue import Queue
from threading import Thread

NUM_WORKER_THREADS = 1

ALL_RESULTS = []

# ----------------------------------------------------------

def find_worker(worker_number):
    
    while not q.empty():
        
        start_time = time.time()
        
        id_to_find = q.get()
        
        if os.path.isfile('/home/data/everything_to_everything/' + id_to_find + '.tsv'):
            pass
        else:
        
            results = subprocess.getoutput('grep ' + id_to_find + ' /mnt/ramdisk/GROUPED.all_shingles.tsv')
            n_found = len(results.strip().split('\n'))

            f = open('/home/data/everything_to_everything/' + id_to_find + '.tsv', 'w', encoding='utf-8')
            f.write(results + '\n')
            f.close()        
        
        q.task_done()

        stop_time = time.time()

        #ALL_RESULTS.append([id_to_find, n_found, (stop_time - start_time)])
    
# ----------------------------------------------------------

q = Queue()

for path_to_file in glob.glob(INPUT_FOLDER + '*.xml')[:100]:
    q.put(path_to_file.split('/')[-1].split('.')[0])
    
total_start_time = time.time()
    
for a in range(NUM_WORKER_THREADS):
    t = Thread(target=find_worker, args=(a,))
    t.daemon = True
    t.start()

q.join()
Quantum vertice ad auras Aethereas, tantum radice ad tartara tendit
print('time:', (time.time() - total_start_time))

print('Done!')

time: 0.09910273551940918
Done!


In [8]:
print(len(ALL_RESULTS))
print(ALL_RESULTS)

100
[['A01475', 5774, 9.163329839706421], ['A13775', 15048, 9.566068887710571], ['A66249', 115, 9.631157159805298], ['B01304', 1704, 9.686990022659302], ['A12494', 1816, 9.753575563430786], ['B04899', 283, 9.786089897155762], ['A32937', 1784, 9.238839387893677], ['A79690', 92, 9.446941375732422], ['A36460', 3447, 9.334395408630371], ['A55158', 31, 9.291319847106934], ['A40317', 851, 9.552822589874268], ['A79165', 32063, 9.9742591381073], ['A82498', 85, 9.304962635040283], ['B08976', 972, 9.544893503189087], ['A73478', 20616, 9.518252611160278], ['A85110', 133, 9.858133316040039], ['A39403', 209, 9.97384786605835], ['A42033', 366, 10.308471441268921], ['B06538', 119, 8.803656339645386], ['A57159', 2292, 9.176780462265015], ['A48245', 710, 9.543368101119995], ['A87142', 473, 10.041020631790161], ['A49346', 236, 9.706344842910767], ['B43656', 23118, 9.94112515449524], ['A41702', 383, 10.06716799736023], ['A89238', 856, 9.271368265151978], ['A83297', 191, 9.311151266098022], ['A93290', 132